# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.83124   
 0.0488543 
 0.89387   
 0.509326  
 0.0496531 
 0.524853  
 0.132509  
 0.568908  
 0.863483  
 0.992563  
 0.248089  
 0.94613   
 0.269561  
 ⋮         
 0.61265   
 0.539291  
 0.456558  
 0.00924826
 0.640047  
 0.0994323 
 0.604606  
 0.684442  
 0.233361  
 0.170325  
 0.681148  
 0.943612  

In [2]:
sum(a)

5.000656519812731e6

In [ ]:
@time sum(a)

@time sum(a)

@time sum(a)


The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [ ]:
#Pkg.add("BenchmarkTools")

In [3]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
c_sum(a)

5.000656519813084e6

In [6]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [7]:
c_sum(a) - sum(a)  

3.5297125577926636e-7

In [8]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [9]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [10]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.183 ms (0.00% GC)
  median time:      9.254 ms (0.00% GC)
  mean time:        9.799 ms (0.00% GC)
  maximum time:     14.556 ms (0.00% GC)
  --------------
  samples:          510
  evals/sample:     1

In [11]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 8.182975 msec


In [12]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 8.18298

In [13]:
using Plots
gr()

Plots.GRBackend()

In [14]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.5 
 
 
 9.0 
 
 
 9.5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [ ]:
#Pkg.add("PyCall")

In [15]:
using PyCall

In [16]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [17]:
pysum(a)

5.000656519813084e6

In [18]:
pysum(a) ≈ sum(a)

true

In [19]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     68.324 ms (0.00% GC)
  median time:      78.533 ms (0.00% GC)
  mean time:        80.912 ms (0.00% GC)
  maximum time:     101.628 ms (0.00% GC)
  --------------
  samples:          62
  evals/sample:     1

In [20]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 8.18298
  "Python built-in" => 68.3235

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [21]:
using Conda 
#Conda.add("numpy")

In [22]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     3.934 ms (0.00% GC)
  median time:      4.282 ms (0.00% GC)
  mean time:        4.508 ms (0.00% GC)
  maximum time:     8.928 ms (0.00% GC)
  --------------
  samples:          1107
  evals/sample:     1

In [23]:
numpy_sum(apy_list) # python thing

5.000656519812735e6

In [24]:
numpy_sum(apy_list) ≈ sum(a)

true

In [25]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 8.18298
  "Python numpy"    => 3.93357
  "Python built-in" => 68.3235

# 4. Python, hand-written 

In [26]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x13d082d70>

In [27]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     456.748 ms (0.00% GC)
  median time:      470.381 ms (0.00% GC)
  mean time:        479.317 ms (0.00% GC)
  maximum time:     517.715 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1

In [28]:
@which sum([1.5])

sum(a) in Base at reduce.jl:359

In [29]:
ccall(:sleep, Void, (Cint,), 3)

In [30]:
Cint, Csize_t

(Int32, UInt64)

In [31]:
sum_py(apy_list)

5.000656519813084e6

In [32]:
sum_py(apy_list) ≈ sum(a)

true

In [33]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 8.18298
  "Python numpy"        => 3.93357
  "Python hand-written" => 456.748
  "Python built-in"     => 68.3235

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [34]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [35]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.744 ms (0.00% GC)
  median time:      3.890 ms (0.00% GC)
  mean time:        4.411 ms (0.00% GC)
  maximum time:     10.411 ms (0.00% GC)
  --------------
  samples:          1132
  evals/sample:     1

In [36]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 8.18298
  "Python numpy"        => 3.93357
  "Python hand-written" => 456.748
  "Python built-in"     => 68.3235
  "Julia built-in"      => 3.74428

# 6. Julia (hand-written) 

In [42]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A   
        s += a   
    end
    s
end

mysum (generic function with 1 method)

In [46]:
α²+π

π = 3.1415926535897...

In [38]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.149 ms (0.00% GC)
  median time:      11.441 ms (0.00% GC)
  mean time:        11.341 ms (0.00% GC)
  maximum time:     17.139 ms (0.00% GC)
  --------------
  samples:          441
  evals/sample:     1

In [39]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 8.18298
  "Python numpy"        => 3.93357
  "Julia hand-written"  => 8.14899
  "Python hand-written" => 456.748
  "Python built-in"     => 68.3235
  "Julia built-in"      => 3.74428

# Summary

In [40]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C........................8.2
Julia built-in...........3.7
Julia hand-written.......8.1
Python built-in.........68.3
Python hand-written....456.7
Python numpy.............3.9


In [41]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............3.74
Python numpy..............3.93
Julia hand-written........8.15
C.........................8.18
Python built-in..........68.32
Python hand-written.....456.75
